# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

In [3]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import seaborn
%matplotlib inline

In [4]:
# Make a request
url_main = 'https://www.topuniversities.com'  # Found with postman
r = requests.get(url_main + '/sites/default/files/qs-rankings-data/357051.txt')
print('Response status code: {0}\n'.format(r.status_code))

page_body = r.text

# Serialize the json data with json library
rank_json = json.loads(page_body)

Response status code: 200



In [5]:
rank_df = pd.DataFrame()
rank_df = rank_df.from_dict(rank_json['data']).head(200)
rank_df.stars
rank_df.drop(['logo', 'stars', 'nid','cc', 'score'], axis=1, inplace=True)
rank_df.set_index('core_id', inplace=True)
rank_df = rank_df[['title', 'rank_display', 'country', 'region', 'url']]
rank_df

,title,rank_display,country,region,url
core_id,,,,,
410,Massachusetts Institute of Technology (MIT),1,United States,North America,/universities/massachusetts-institute-technolo...
573,Stanford University,2,United States,North America,/universities/stanford-university
253,Harvard University,3,United States,North America,/universities/harvard-university
94,California Institute of Technology (Caltech),4,United States,North America,/universities/california-institute-technology-...
95,University of Cambridge,5,United Kingdom,Europe,/universities/university-cambridge
478,University of Oxford,6,United Kingdom,Europe,/universities/university-oxford
365,UCL (University College London),7,United Kingdom,Europe,/universities/ucl-university-college-london
356,Imperial College London,8,United Kingdom,Europe,/universities/imperial-college-london
120,University of Chicago,9,United States,North America,/universities/university-chicago


In [6]:
# Find the values for number of faculty members and students
fac_memb_tot = []
fac_memb_int = []
nb_stud_tot = []
nb_stud_int = []

for url in rank_df.url:
    
    uni_url = requests.get(url_main + url)
    uni_body = uni_url.text
    
    # 1) Faculty Members: 
    # 1.1) Total
    soup = BeautifulSoup(uni_body, 'html.parser')
    soup1 = soup.find('div', class_='total faculty')
    if soup1:
        soup2 = soup1.find('div', class_='number')
        if soup2:
            fac_memb_tot.append({'fac_memb_tot': soup2.text})
        else:
            fac_memb_tot.append({'fac_memb_tot': -99})
    else:
        fac_memb_tot.append({'fac_memb_tot': -99})
        
    # 1.2) International
    soup3 = soup.find('div', class_='inter faculty')
    if soup3:
        soup4 = soup3.find('div', class_='number')
        if soup4:
            fac_memb_int.append({'fac_memb_int': soup4.text})
        else:
            fac_memb_int.append({'fac_memb_int': -99})
    else:
        fac_memb_int.append({'fac_memb_int': -99})
        
    # 2) Students:
    # 2.1) Total
    soup5 = soup.find('div', class_='total student')
    if soup5:
        soup6 = soup5.find('div', class_='number')
        if soup6:
            nb_stud_tot.append({'nb_stud_tot': soup6.text})
        else:
            nb_stud_tot.append({'nb_stud_tot': -99})
    else:
        nb_stud_tot.append({'nb_stud_tot': -99})
    
    # 2.2) International
    soup7 = soup.find('div', class_='total inter')
    if soup7:
        soup8 = soup7.find('div', class_='number')
        if soup8:
            nb_stud_int.append({'nb_stud_int': soup8.text})
        else:
            nb_stud_int.append({'nb_stud_int': -99})
    else:
        nb_stud_int.append({'nb_stud_int': -99})

In [7]:
# Convert the list to dataframe
fac_memb_tot1 = pd.DataFrame.from_dict(fac_memb_tot).replace({r'\n': ''}, regex=True).replace({r',': ''}, regex=True).apply(pd.to_numeric).astype(int)
            
fac_memb_int1 = pd.DataFrame.from_dict(fac_memb_int).replace({r'\n': ''}, regex=True).replace({r',': ''}, regex=True).apply(pd.to_numeric).astype(int)

nb_stud_tot1 = pd.DataFrame.from_dict(nb_stud_tot).replace({r'\n': ''}, regex=True).replace({r',': ''}, regex=True).apply(pd.to_numeric).astype(int)

nb_stud_int1 = pd.DataFrame.from_dict(nb_stud_int).replace({r'\n': ''}, regex=True).replace({r',': ''}, regex=True).apply(pd.to_numeric).astype(int)

In [8]:
# Concatenate the different dataframes
details_df = pd.concat([fac_memb_tot1, fac_memb_int1, nb_stud_tot1, nb_stud_int1], axis=1)
details_df.set_index(rank_df.index, inplace=True)
qs_df = pd.concat([rank_df, details_df], axis=1)
qs_df.drop(['url'], axis=1, inplace=True)

# Delete the = in rank_display
qs_df.rank_display = qs_df.rank_display.replace({r'=': ''}, regex=True).apply(pd.to_numeric).astype(int)

# Create the faculty members ratio and number of students ratio
qs_df['fac_memb_ratio'] = qs_df.fac_memb_tot / qs_df.nb_stud_tot
qs_df['int_stud_ratio'] = qs_df.nb_stud_int / qs_df.nb_stud_tot
qs_df

# To do, treat -99 (unknowns)

,title,rank_display,country,region,fac_memb_tot,fac_memb_int,nb_stud_tot,nb_stud_int,fac_memb_ratio,int_stud_ratio
core_id,,,,,,,,,,
410,Massachusetts Institute of Technology (MIT),1,United States,North America,2982,1679,11067,3717,0.269450,0.335863
573,Stanford University,2,United States,North America,4285,2042,15878,3611,0.269870,0.227422
253,Harvard University,3,United States,North America,4350,1311,22429,5266,0.193945,0.234785
94,California Institute of Technology (Caltech),4,United States,North America,953,350,2255,647,0.422616,0.286918
95,University of Cambridge,5,United Kingdom,Europe,5490,2278,18770,6699,0.292488,0.356899
478,University of Oxford,6,United Kingdom,Europe,6750,2964,19720,7353,0.342292,0.372870
365,UCL (University College London),7,United Kingdom,Europe,6345,2554,31080,14854,0.204151,0.477928
356,Imperial College London,8,United Kingdom,Europe,3930,2071,16090,8746,0.244251,0.543567
120,University of Chicago,9,United States,North America,2449,635,13557,3379,0.180645,0.249244


In [9]:
#Before doing the exploratory analysis, a analysis of the different columns we are seeing.
qs_df.describe()


,rank_display,fac_memb_tot,fac_memb_int,nb_stud_tot,nb_stud_int,fac_memb_ratio,int_stud_ratio
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,100.245000,2897.890000,777.555000,25580.090000,5052.375000,0.130324,0.214699
std,57.771991,2016.958915,669.185642,17189.592558,3931.861798,0.086350,0.129023
min,1.000000,-99.000000,-99.000000,-99.000000,-99.000000,0.040391,0.009892
25%,50.750000,1603.000000,322.750000,14831.250000,2434.500000,0.084675,0.122241
50%,100.500000,2602.500000,591.500000,23738.500000,4248.000000,0.115368,0.194800
75%,149.500000,3627.250000,1027.250000,32205.500000,6877.250000,0.148743,0.280678
max,201.000000,16421.000000,3905.000000,141939.000000,27109.000000,1.000000,1.000000


In [15]:
qs_df.fac_memb_tot.value_counts()[-99]

1

In [ ]:
qs_df.drop(-99)